## Data Pull


In [29]:
import os
import yfinance as yf
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import coint
from statsmodels.tsa.vector_ar.vecm import coint_johansen
import os

basket_1 = ["SNOW", "DDOG", "NET", "ESTC", "MDB"]

# Basket 2: Cloud Security (Next-Gen)
basket_2 = ["CRWD", "ZS", "PANW", "OKTA", "S"]

# Basket 3: Enterprise SaaS Platforms
basket_3 = ["CRM", "NOW", "WDAY", "TEAM", "HUBS"]

# Basket 4: Ad Tech (Programmatic)
basket_4 = ["TTD", "TRADE", "MGNI", "PUBM", "RAMP"]

# Basket 5: Analog Semiconductors
basket_5 = ["TXN", "ADI", "MCHP", "ON", "NXPI"]

# Basket 6: GPU/AI Accelerators
basket_6 = ["AMD", "NVDA", "AVGO", "MRVL", "QCOM"]

# Basket 7: Communication/Collaboration SaaS
basket_7 = ["ZM", "TWLO", "DOCU", "RNG", "FIVN"]

# Basket 8: Payments/Fintech
basket_8 = ["PYPL", "SQ", "BILL", "SHOP", "MELI"]

# Basket 9: DevOps/Developer Tools
basket_9 = ["GTLB", "FROG", "PD", "DOCN", "CFLT"]

# Basket 10: Networking Infrastructure
basket_10 = ["CSCO", "ANET", "CIEN", "JNPR", "VMW"]

# Basket 11: E-commerce Platforms (weaker grouping)
basket_11 = ["EBAY", "WIX", "BIGC", "AMZN", "U"]

# Basket 12: Security (Identity/Access)
basket_12 = ["SAIL", "VRNS", "CYBR", "TENB", "CHKP"]

# Basket 13: Semiconductor Equipment (MPWR doesn't fit perfectly)
basket_13 = ["LRCX", "KLAC", "AMAT", "ENTG", "MPWR"]

# Basket 14: Social/Consumer Content
basket_14 = ["META", "PINS", "SNAP", "SPOT", "ROKU"]

# Basket 15: Productivity/Work Management (weakest grouping)
basket_15 = ["ASAN", "SMAR", "DOMO", "VEEV", "PCTY"]

# All baskets in a list
all_baskets = [
    basket_1, basket_2, basket_3, basket_4, basket_5,
    basket_6, basket_7, basket_8, basket_9, basket_10,
    basket_11, basket_12, basket_13, basket_14, basket_15
]


In [30]:
import numpy as np

def get_price_data(tickers, period="1y", max_missing=25, log_prices=True):
           """Download and clean stock price data, save to CSV files"""
           try:
               print(f"Downloading data for {len(tickers)} stocks...")
               data = yf.download(tickers, period=period)

               # Clean missing data
               close_missing = data["Close"].isnull().sum()
               stocks_to_drop = close_missing[close_missing > max_missing].index.tolist()

               if stocks_to_drop:
                   print(f"Dropping {len(stocks_to_drop)} stocks with >{max_missing} missing values")
                   data = data.drop(stocks_to_drop, axis=1, level=1)

               # Save files
               data.to_csv("data.csv")
               # Simple summary
               remaining_stocks = len(data.columns.get_level_values(1).unique())
               print(f"Final dataset: {remaining_stocks} stocks")

               return data

           except Exception as e:
               print(f"Error downloading data: {e}")
               return None

clean_data = get_price_data(basket_1, max_missing=25)

def get_close_price_data(tickers, period="1y", max_missing=25, log_prices=True):
        try:
            print(f"Downloading close prices for {len(tickers)} stocks...")
            data = yf.download(tickers, period=period)
            close_data = data["Close"]
            missing = close_data.isnull().sum()
            close_data = close_data.drop(missing[missing > max_missing].index, axis=1)
            close_data = close_data.ffill()
        
            if log_prices:
                close_data = np.log(close_data)
        
            filename = "_".join(tickers) + ".csv"
            close_data.to_csv(filename)
        
            print(f"Final dataset: {len(close_data.columns)} stocks")
            return close_data
        
        except Exception as e:
            print(f"Error: {e}")
            return None


close_data = get_close_price_data(basket_1, max_missing=25)
      


C:\Users\mgray\AppData\Local\Temp\ipykernel_35988\1234747663.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, period=period)
[*********************100%***********************]  5 of 5 completed
C:\Users\mgray\AppData\Local\Temp\ipykernel_35988\1234747663.py:34: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, period=period)
[                       0%                       ]

Final dataset: 5 stocks


[*********************100%***********************]  5 of 5 completed

Final dataset: 5 stocks


## Data Retrieval For Each Basket:

In [31]:
''' I need data for each basket, I can run a for loop across my function or make a new one'''

def get_all_close_data(tickers_super_list):
    for baskets in tickers_super_list:
        get_close_price_data(baskets, max_missing=25)

get_all_close_data(all_baskets)
    

C:\Users\mgray\AppData\Local\Temp\ipykernel_35988\1234747663.py:34: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, period=period)


[*********************100%***********************]  5 of 5 completed
C:\Users\mgray\AppData\Local\Temp\ipykernel_35988\1234747663.py:34: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, period=period)
[*******************   40%                       ]  2 of 5 completed

Final dataset: 5 stocks


[*********************100%***********************]  5 of 5 completed
C:\Users\mgray\AppData\Local\Temp\ipykernel_35988\1234747663.py:34: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, period=period)
[                       0%                       ]

Final dataset: 5 stocks


...

Final dataset: 5 stocks


[*********************100%***********************]  5 of 5 completed
C:\Users\mgray\AppData\Local\Temp\ipykernel_35988\1234747663.py:34: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, period=period)
[**********************80%*************          ]  4 of 5 completed

Final dataset: 5 stocks


[*********************100%***********************]  5 of 5 completed

1 Failed download:
['SMAR']: YFPricesMissingError('possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")')


Final dataset: 4 stocks


## Johansen Cointegration Testing On Baskets:

In [ ]:

def load_all_csv(folder_path="."):
    '''load all CSVs from folder into a dictionary of DataFrames'''
    dfs = {}
    for file in os.listdir(folder_path):
        if file.endswith('.csv') and '_' in file and file != 'data.csv':
            basket_name = file.replace('.csv', '')
            df = pd.read_csv(os.path.join(folder_path, file), index_col=0)
            dfs[basket_name] = df
    return dfs

all_data = load_all_csv()


def run_johansen(df):
    """Run Johansen cointegration test on a basket"""
    result = coint_johansen(df, det_order=0, k_ar_diff=1)
    
    print("Trace Statistics:", result.lr1)
    print("Critical Values (95%):", result.cvt[:, 1])
    print("\nCointegrating relationships:", sum(result.lr1 > result.cvt[:, 1]))
    
    return result


def test_all_baskets(all_data, crit_level=0):
    """
    crit_level: 0=90%, 1=95%, 2=99%
    """
    for name, df in all_data.items():
        print(f"\n{name}")
        result = coint_johansen(df, det_order=0, k_ar_diff=5)
        n_coint = sum(result.lr1 > result.cvt[:, crit_level])
        print(n_coint)

test_all_baskets(all_data)


import matplotlib.pyplot as plt

def test_cointegration(tickers, period="1y", crit_level=1, k_ar_diff=5):
    """
    Test cointegration and plot results
    
    Parameters:
    tickers: list of stock tickers
    period: time frame (e.g., "6mo", "1y", "2y")
    crit_level: 0=90%, 1=95%, 2=99%
    k_ar_diff: lag length (default=5)
    """
    # Download log price data
    print(f"Downloading data for {tickers}...")
    data = yf.download(tickers, period=period, auto_adjust=True)
    log_prices = np.log(data["Close"]).ffill().dropna()
    
    # Run Johansen test
    result = coint_johansen(log_prices, det_order=0, k_ar_diff=k_ar_diff)
    
    # Get cointegration rank
    coint_rank = sum(result.lr1 > result.cvt[:, crit_level])
    
    # Print results
    crit_labels = {0: "90%", 1: "95%", 2: "99%"}
    print(f"\n{'='*50}")
    print(f"Cointegration Test Results at {crit_labels[crit_level]} confidence")
    print(f"{'='*50}")
    print(f"Cointegration Rank: {coint_rank}")
    print(f"Trace Statistics: {result.lr1}")
    print(f"Critical Values: {result.cvt[:, crit_level]}")
    
    # Print cointegrating vectors if they exist
    if coint_rank > 0:
        print(f"\nCointegrating Vectors:")
        print(result.evec[:, :coint_rank])
    else:
        print("\nNo cointegrating relationships found")
    
    # Plot log prices
    plt.figure(figsize=(12, 6))
    for col in log_prices.columns:
        plt.plot(log_prices.index, log_prices[col], label=col)
    plt.title(f"Log Prices - {', '.join(tickers)}")
    plt.xlabel("Date")
    plt.ylabel("Log Price")
    plt.legend()
    plt.grid(True)
    plt.show()
    
    # Return cointegrating vectors if they exist
    return result.evec[:, :coint_rank] if coint_rank > 0 else None
